In [ ]:
import json

import numpy as np
import tqdm as notebook_tqdm
from sentence_transformers import SentenceTransformer
from sklearn.cluster import HDBSCAN

In [ ]:
def conjoin_signatures_semantic(
    registry_output: dict,
    embedding_model: str = "all-MiniLM-L6-v2",
    cache_dir: str = "./.models",
):
    hashes = list(registry_output.keys())
    if not hashes:
        return {}

    signatures_as_text = []
    for h in hashes:
        h_dict = dict(registry_output[h]["signature"])
        # remove the 'black hole' field that swallows everything
        h_dict.pop("_unparsed", None)

        # sort keys to ensure structural identity regardless of log order
        sorted_keys = sorted(h_dict.keys())

        if not sorted_keys:
            text_rep = "schema:empty_blob"
        else:
            # 'field:' prefix to define the role of the tokens
            text_rep = " ".join([f"field:{k}" for k in sorted_keys])

        signatures_as_text.append(text_rep)

    model = SentenceTransformer(embedding_model, cache_folder=cache_dir)
    embeddings = model.encode(signatures_as_text)
    X = np.ascontiguousarray(embeddings, dtype=np.float64)

    clusterer = HDBSCAN(
        min_cluster_size=2,
        min_samples=1,
        metric="cosine",
        cluster_selection_epsilon=0.08,
        cluster_selection_method="eom",
        allow_single_cluster=True,
    )

    labels = clusterer.fit_predict(X)

    conjoined_map = {}
    for i, cluster_id in enumerate(labels):
        h = hashes[i]
        # unique IDs to outliers so they don't group into one '-1' bucket
        final_id = int(cluster_id) if cluster_id != -1 else (400 + i)

        conjoined_map[h] = {
            "cluster_id": final_id,
            "keys": list(registry_output[h]["signature"].keys()),
            "is_outlier": cluster_id == -1,
        }

    return conjoined_map

In [ ]:
with open("sample_registry.json") as f:
    registry = json.load(f)

registry

In [ ]:
cluster_mapping = conjoin_signatures_semantic(registry)

group_by_cluster = {}
for k, v in cluster_mapping.items():
    cluster_id = v["cluster_id"]
    keys = v["keys"]
    is_outlier = bool(v["is_outlier"])
    group_by_cluster[cluster_id] = group_by_cluster.get(cluster_id, [])
    group_by_cluster[cluster_id].append(
        {"signature_hash": k, "fields": keys, "is_outlier": is_outlier}
    )


print(f"Semantic Clusters: \n{json.dumps(group_by_cluster, indent=4)}\n")